In [1]:
# befOre using OPTFLOW package to calculate optical flow --> install below openCV package

# pip install opencv-contrib-python 

import os
import numpy as np
import cv2
from glob import glob
from multiprocessing import Pool

import matplotlib.pyplot as plt

In [2]:
_IMAGE_SIZE = 256
MAX_SEQ_LENGTH = 8

In [3]:
video_path = "../vid_data/train/catch/Ball_hochwerfen_-_Rolle_-_Ball_fangen_(Timo_3)_catch_f_cm_np1_ri_med_1.avi"

In [4]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(video_path, max_frames=0, resize=(_IMAGE_SIZE, _IMAGE_SIZE)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    frames=frames[:MAX_SEQ_LENGTH]
    return frames

In [5]:
def compute_TVL1(prev, curr, bound=15):
    """Compute the TV-L1 optical flow."""
    TVL1 = cv2.optflow.DualTVL1OpticalFlow.create()
    # TVL1 = cv2.createOptFlow_DualTVL1()
    flow = TVL1.calc(prev, curr, None)
    assert flow.dtype == np.float32

    flow = (flow + bound) * (255.0 / (2*bound))
    flow = np.round(flow).astype(int)
    flow[flow >= 255] = 255
    flow[flow <= 0] = 0

    return flow

In [6]:
def cal_for_frames(path):
    frames = load_video(path)
    frames = np.array(frames)
    print(frames.shape)
    # print(frames[5])
    # frames.sort()

    flow = []
    prev = frames[0]
    prev = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    for i, frame_curr in enumerate(frames):
        curr = frame_curr
        curr = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
        tmp_flow = compute_TVL1(prev, curr)
        # tmp_flow = cv2.calcOpticalFlowFarneback(prev, curr, None, 0.8, 3, 10, 3, 7, 1.1, cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
        flow.append(tmp_flow)
        prev = curr

    return flow


flow = np.array(cal_for_frames(video_path))
print(flow.shape)
# print(flow[5])


(8, 256, 256, 3)
[[[ 53  75  49]
  [ 49  71  45]
  [ 49  71  43]
  ...
  [ 63  87  69]
  [ 55  80  62]
  [ 60  85  67]]

 [[ 52  74  48]
  [ 51  73  47]
  [ 51  73  45]
  ...
  [ 65  89  72]
  [ 60  85  67]
  [ 65  90  72]]

 [[ 49  71  45]
  [ 49  71  45]
  [ 51  73  45]
  ...
  [ 77 100  83]
  [ 68  93  75]
  [ 70  95  77]]

 ...

 [[125 155  78]
  [125 155  78]
  [125 154  80]
  ...
  [118 153  79]
  [124 151  79]
  [124 151  79]]

 [[125 156  75]
  [125 156  75]
  [125 155  76]
  ...
  [116 153  78]
  [122 151  78]
  [123 152  79]]

 [[124 155  74]
  [125 156  75]
  [125 155  77]
  ...
  [116 153  78]
  [122 151  78]
  [122 151  78]]]
(8, 256, 256, 2)
[[[ 91 126]
  [ 91 126]
  [ 91 126]
  ...
  [123  85]
  [123  85]
  [123  85]]

 [[ 91 126]
  [ 91 126]
  [ 91 126]
  ...
  [123  85]
  [123  85]
  [123  85]]

 [[ 91 126]
  [ 91 126]
  [ 91 126]
  ...
  [123  85]
  [123  85]
  [123  85]]

 ...

 [[ 80 177]
  [ 80 177]
  [ 80 177]
  ...
  [ 94 204]
  [ 95 204]
  [ 95 204]]

 [[ 80 177